In [ ]:
import os

from matplotlib import ticker
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from mpl_toolkits.mplot3d.art3d import pathpatch_2d_to_3d, Poly3DCollection
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.viz import (set_colorblind, fig_config, set_axes_equal,
                              save_fig)
from helpers import (clean_df, export_pcd, export_fields, poynting_vector,
                     export_rect_idx, get_imcolors)

In [ ]:
set_colorblind()
%config InlineBackend.figure_format = 'retina'

# CAD model

In [ ]:
# input data

polarization = 'te'
f = 26

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
pcd.paint_uniform_color([0.5, 0.5, 0.5])
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=9, origin=center+np.array([6, -25, -20])
)
pcd.estimate_normals()
radii = [0.005, 0.01, 0.02, 0.04]
rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector(radii)
)

In [ ]:
view_config = {
    'zoom': 0.6,
    'front': [0.99757547016811021, -0.0053197730130150481, 0.069389345968699973],
    'lookat': [73.970504527733638, 21.162127098201477, -9.8491774343806799],
    'up': [0.0062516025708518452, 0.99989308520692732, -0.01321876017899459],
}
color = get_imcolors([pcd, rec_mesh], view_config)
set_colorblind()
fig_config()
fig = plt.figure()
ax = fig.add_subplot()
ax.imshow(color, origin='upper')

fname = os.path.join('figures', 'ear_model')
save_fig(fig, fname=fname, formats=['png'])

# Parametrized averaging surface projection onto the model

In [ ]:
# input data

polarization = 'te'
f = 26
edge_length = 20
origin = [-25, 15]

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]
rect_center = [origin[0] + edge_length / 2, origin[1] + edge_length / 2]
_, rect_idx = export_rect_idx(xyz_zy, rect_center, edge_length, view='zy')

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.2, color='red')
# handle for legend
rect_handle = Rectangle([0,0], 0, 0, color='red', ec='None', alpha=0.8,
                        label='averaging\nsurface')

# averaging surface(s)
rect_pos = xyz[:, 0].max()
origin = [-25, 15]
edge_length = 20
param_rect = Rectangle(origin, edge_length, edge_length,
                       ec='black', fc='None', lw=2,
                       label='parametric\nsurface')
ax.add_patch(param_rect)
pathpatch_2d_to_3d(param_rect, z=100, zdir='y')

# additional figure configuration
ax.set(xlabel='$z$ [mm]', ylabel='$x$ [mm]',
       xticks=[10, -10, -30],
       yticks=[50, 70, 90],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 12
ax.zaxis.labelpad = 12
ax.view_init(15, 70)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
fig.legend(handles=[rect_handle, param_rect],
           bbox_to_anchor=(0.75, 1),
           handlelength=1, handleheight=1,
           ncol=2)
fig.tight_layout();

# save figure
fname = os.path.join('figures', 'projection_a')
save_fig(fig, fname=fname, formats=['png'])

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud
ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
           s=0.25, alpha=0.02, color='gray')
ax.scatter(xyz_zy[rect_idx, 2], xyz_zy[rect_idx, 0], xyz_zy[rect_idx, 1],
           s=0.25, alpha=0.15, color='red')

# averaging surface(s)
rect_pos = xyz[:, 0].max()
origin = [-25, 15]
edge_length = 20
param_rect = Rectangle(origin, edge_length, edge_length,
                       ec='black', fc='None', lw=2,
                       label='parametrized\nsurface')
ax.add_patch(param_rect)
pathpatch_2d_to_3d(param_rect, z=100, zdir='y')

# additional figure configuration
ax.set(xlabel='$z$ [mm]',
       xticks=[10, -10, -30],
       yticks=[],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 20
ax.tick_params(axis='z', pad=10)
ax.view_init(0, 90)
ax.set_box_aspect([1, 1, 1])
ax.set_proj_type('ortho')
ax = set_axes_equal(ax)
fig.tight_layout();

# save figure
fname = os.path.join('figures', 'projection_b')
save_fig(fig, fname=fname, formats=['png'])

# APD distribution

In [ ]:
origin_ds = {'te26_4': [-25.28, 17.25],
             'tm26_4': [-19.77, 5.22],
             'te60_4': [-25.09, 19.09],
             'te60_1': [-8.50, 11.38],
             'tm60_4': [-13.24, 13.05],
             'tm60_1': [-10.32, 13.46],
             }

In [ ]:
# input data

polarization = 'tm'
f = 26
edge_length = 20

In [ ]:
if edge_length == 20:
    A = 4
    origin = origin_ds[f'{polarization}{f}_{A}']
elif edge_length == 10:
    A = 1
    origin = origin_ds[f'{polarization}{f}_{A}']
else:
    raise ValueError(f'Edge length {edge_length} is not defined.')

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)

# postprocessing on the surface of the model
E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)
pcd.estimate_normals()
n = np.asarray(pcd.normals)
APD = np.abs(Sx.real * n[:, 0] + Sy.real * n[:, 1] + Sz.real * n[:, 2])

In [ ]:
skip = 1
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')

# point cloud and apd spatial distribution
cs = ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
                c=APD[::skip],
                cmap='viridis',
                s=0.5,
               )

# averaging surface(s)
rect_pos = xyz[:, 0].max()
if (f == 60) and (edge_length == 20):
    patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=2)
    origin_red = [origin[0] + edge_length / 4,
                  origin[1] + edge_length / 4]
    patch_rect_red = Rectangle(origin_red, edge_length / 2, edge_length / 2, fc='None', ls='--', lw=2)
    ax.add_patch(patch_rect)
    pathpatch_2d_to_3d(patch_rect, z=rect_pos, zdir='y')
    ax.add_patch(patch_rect_red)
    pathpatch_2d_to_3d(patch_rect_red, z=rect_pos, zdir='y')
else:
    patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=2)
    ax.add_patch(patch_rect)
    pathpatch_2d_to_3d(patch_rect, z=80, zdir='y')
    
    # no colorbar for f = 60 GHz and A = 4 cm^2
    cbar = fig.colorbar(cs, shrink=0.55, pad=-0.05)
    tick_locator = ticker.MaxNLocator(nbins=4)
    cbar.locator = tick_locator
    cbar.update_ticks()
    # label only for f = 60 GHz and A = 1 cm^1
    if (f == 60) and (edge_length == 10):
        cbar.ax.set_ylabel('$PD$ [W/m$^{2}$]')
    cbar.ax.yaxis.labelpad = 12

# additional figure configuration
ax.set(xlabel='$z$ [mm]', ylabel='$x$ [mm]',
       xticks=[10, -10, -30],
       #yticks=[50, 70, 90],
       yticks=[],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 10
ax.zaxis.labelpad = 20
ax.tick_params(axis='z', pad=10)
ax.view_init(1, 90)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
fig.tight_layout();

# save figure
fname = os.path.join('figures', f'{polarization}{f}_{A}')
save_fig(fig, fname=fname, formats=['png'])

# SAR distribution

In [ ]:
# input data

polarization = 'te'
f = 26

In [ ]:
# load model data

fname_block = os.path.join('data', f'loss_3D_ear_{f}GHz_{polarization.upper()}_front.txt')
df = (pd.read_csv(fname_block, names=['x [mm]', 'y [mm]', 'z [mm]', 'Pl [W/m^3]'],
                  header=None, delim_whitespace=True, skiprows=[0, 1])
      [lambda x: (x['Pl [W/m^3]'] != 0.0)])
df.reset_index(drop=True, inplace=True)
xyz = export_pcd(df)
Pl = df['Pl [W/m^3]'].to_numpy()
tissue_density = 1049

# postprocessing
SAR = Pl / tissue_density

In [ ]:
skip = 2
scaler = 2
fig_config(latex=True, text_size=20, scaler=scaler)
fig = plt.figure()
ax = plt.axes(projection ='3d')
cs = ax.scatter(xyz[::skip, 2], xyz[::skip, 0], xyz[::skip, 1],
                c=SAR[::skip],
                cmap='viridis',
                s=0.5,
               )
cbar = fig.colorbar(cs, shrink=0.55, pad=0)
cbar.ax.set_ylabel('$SAR$ [W/kg]')
cbar.ax.yaxis.labelpad = 12
ax.set(xlabel='$z$ [mm]', ylabel='$x$ [mm]',
       xticks=[10, -10, -30],
       yticks=[50, 70, 90],
       zticks=[0, 20, 40])
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('$y$ [mm]', rotation=90)
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.labelpad = 15
ax.yaxis.labelpad = 15
ax.zaxis.labelpad = 12
ax.view_init(10, 70)
ax.set_box_aspect([1, 1, 1])
ax = set_axes_equal(ax)
fig.tight_layout();

# save figure
#fname = os.path.join('figures', f'{polarization}{f}_sar')
#save_fig(fig, fname=fname, formats=['png'])